# Matching 3D coordinates, with transformations

- <https://adventofcode.com/2021/day/19>

We are asked to figure out how many different beacons the scanners can 'see', by matching overlapping beacon coordinates. The trick is to know, without spending too much computing resources on this, when two sets of scanners are likely to be referencing the same set of beacons.

To do this, I pre-calculate the distances between each pair of beacons in a cloud; this is easily achieved by using the [`scipy.spacial.distance.pdist()` function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html). A cloud of $n$ beacons can be paired up into $n(n-1)/2$ pairs, and the distances between these pairs remains constant no matter what rotation you apply. You can then _intersect_ the sets of distances for two scanners, and see if _enough_ of those distances match. For two scanners to have 12 beacons in common, you need at least $12(12 - 1)/2 = 66$ distances matching.

Once you have determined that two scanners do overlap (have at least 66 distances in common), we need to figure out the correct rotation for the second scanner. I used numpy vectorisation here; a single matrix of rotation transformations is used to produce all 24 possible rotations of the beacon cloud, with a dot product operation (a single `@` matrix multiplication). You can then try each of the beacons involved with one of the matching distances (we don't know which of these will match with which beacon in the other scanner, but their number is limited to 2 or 4 or so), and see if shifting the rotated beacon positions leads to a match.

Determining which of the 24 orientations has a match means we need to figure out how many beacon positions are the same; the orientation with the most matches (provided there are at least 12), is the correct rotation. I used [`numpy.unique()`](https://numpy.org/doc/stable/reference/generated/numpy.unique.html) to generate an array of all unique vectors in both the beacon matrix of the scanner we are trying to match against and all rotations of the beacon cloud, together with the _inverse index_, an array where each position is the index into the unique values array for each input vector. You can then create two [sparse boolean matrices](https://docs.scipy.org/doc/scipy/reference/sparse.html) where rows and columns represent vectors in the input and the indices of the unique values, one for the fixed scanner, the other for all orientations of the scanner we are matching. A `True` value in any given cell connects a vector in one of the beacon clouds to one of the unique values, so representing the _set memberships_ of each cloud. If you then produce the _dot product_ of these two sparse matrices, you essentially create their intersection, and you can then sum this, per rotation, to get a count of intersections. Using [`numpy.argmax()`](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) on this gives us the index of the rotation that matches. With the right orientation, and a known distance, you can then use the updated scanner object to help locate the other scanners.

To position all scanners, you start with a list with a single positioned scanner (it doesn't actually matter which one). I then use a [dequeu](https://docs.python.org/3/library/collections.html#collections.deque) (double-ended queue) for all scanners without a definite positon, and as long as there are scanners in the queue, take the next one and test that one against all positioned scanners, one by one, until I find a match. If matched, it is added to the positioned scanners list, otherwise, if there was no match against any of the positioned scanners, the tested scanner is added to the back of the queue again. Once they all have a position, we only have to count the unique beacon vectors.

And because everything is achieved with vectorised array operations, the whole thing takes milliseconds to run.


In [1]:
from __future__ import annotations
from collections import deque
from dataclasses import dataclass
from functools import cached_property
from itertools import combinations, permutations
from typing import Final
import math

import numpy as np
from scipy.spatial.distance import pdist
from scipy.sparse import lil_matrix


MIN_BEACONS_IN_COMMON: Final[int] = 12
MIN_COMMON_DISTANCES: Final[int] = math.comb(MIN_BEACONS_IN_COMMON, 2)


# Generate the 24 unique 3D rotation matrices
def _rotations() -> np.array:
    # the identity transformation matrix, [1 0 0 0] [0 1 0 0] [0 0 1 0] [0 0 0 1]
    eye = np.identity(4, dtype=np.int8)
    # all permutations of 0, 1 and 2, padded with 3 to keep the bottom eye row in place.
    # used to re-arrange the rows of the eye matrix
    rows = np.pad(
        np.array(list(permutations(range(3)))), ((0, 0), (0, 1)), constant_values=3
    )
    # the product of (-1, 1), times 3, with a 1 added to the end; these are the
    # signs for each row of the rotation matrix.
    signs = np.pad(
        np.array([-1, 1])[
            np.stack(np.meshgrid(*([np.arange(2)] * 3)), axis=-1).reshape(-1, 3)
        ],
        ((0, 0), (0, 1)),
        constant_values=1,
    )
    # produce the product of signs and rows.
    signs, rows = np.repeat(signs, rows.shape[0], axis=0), np.tile(
        rows, (signs.shape[0], 1)
    )
    # lower half of the transformation matrix, used to calculate permutation parity
    # see https://en.wikipedia.org/wiki/Parity_of_a_permutation
    tx, ty = np.tril_indices(3, -1)
    rowsparity = np.prod(np.sign(rows[:, tx] - rows[:, ty]), axis=-1)
    signsparity = np.prod(signs[:, :3], axis=1)
    # all signs and all permutations with the same parity
    signs, rows = signs[rowsparity == signsparity], rows[rowsparity == signsparity]
    count = signs.shape[0]
    # alter eye with the signs combo (cols 0-3), then permute the rows (cols 4-7)
    return (signs[:, :, None] * np.tile(eye, (count, 1)).reshape(-1, 4, 4))[
        np.arange(count)[:, None], rows
    ]


ROTATIONS: Final[np.array] = _rotations()
NOR: int = ROTATIONS.shape[0]


@dataclass
class Scanner:
    beacons: np.array
    position: np.array = np.zeros(3, dtype=np.int16)

    @classmethod
    def from_lines(cls, lines: list[str]) -> Scanner:
        return Scanner(np.genfromtxt(lines, delimiter=",", dtype=np.int16))

    @cached_property
    def distances(self) -> dict[np.float64, set[int]]:
        """Map from distance to pair of indices of beacons"""
        map, combos = {}, combinations(range(self.beacons.shape[0]), 2)
        for dist, pair in zip(pdist(self.beacons), combos):
            try:
                map[dist].update(pair)
            except KeyError:
                map[dist] = set(pair)
        return map

    @cached_property
    def orientations(self) -> np.array:
        # pad with 1s, apply the rotations transformation, then un-pad.
        b = self.beacons
        aug = np.concatenate((b, np.ones((*b.shape[:-1], 1), dtype=b.dtype)), axis=-1)
        return (aug @ ROTATIONS)[..., :-1]

    def __and__(self, other: Scanner) -> Scanner | None:
        """Check for scanner overlap

        Returns new Scanner at correct rotation with position updated, relative to
        other.

        """
        # how many distances are the same? If enough match, there is overlap
        shared = self.distances.keys() & other.distances.keys()
        if sum(len(self.distances[d]) // 2 for d in shared) < MIN_COMMON_DISTANCES:
            return None
        # track some of the sizes involved, number of other and self beacons and
        # orientations (reposititioned)
        nob = other.beacons.shape[0]
        nsb = self.beacons.shape[0]
        nsr = nsb * NOR
        # pick one of the beacons from other that we know has distances in common
        distance = next(iter(shared))
        reference = other.beacons[next(iter(other.distances[distance]))]
        own_pairs = self.distances[distance]
        # try all ends of the matching pairs in self; we don't know what side
        # matches with the reference beacon.
        for i in own_pairs:
            offsets = reference - self.orientations[:, i]
            repositioned = self.orientations + offsets[:, None, :]
            # find unique vectors, and their inverse index, used to quantify
            # how many vectors in a repositioned oriention fit.
            values, ix = np.unique(
                np.vstack((other.beacons, repositioned.reshape(-1, 3))),
                axis=0,
                return_inverse=True,
            )
            if values.shape[0] > nsr + nob - 12:
                continue  # not enough overlap between target beacons and repositioned

            # find the matching orientation intersecting the beacons and repositioned
            # matrix as sets; we count the unique values and create boolean matrices
            # mapping vector values to their index in the beacons and repositioned
            # matrices, then taking the dot product of these two mappings.
            ix_beacons, ix_repos = ix[:nob], ix[nob:]
            obvmembers = lil_matrix((nob, values.shape[0]), dtype=bool)
            obvmembers[np.arange(nob), ix_beacons] = True
            rvmembers = lil_matrix((nsr, values.shape[0]), dtype=bool)
            rvmembers[np.arange(nsr), ix_repos] = True
            matches = (obvmembers.tocsr() @ rvmembers.tocsr().T).T.sum(axis=-1)
            counts = matches.reshape(-1, nsb).sum(axis=-1)
            if not np.any(counts >= MIN_BEACONS_IN_COMMON):
                continue

            # orientation determined, get the corrected beacon positions
            orientation = np.argmax(counts)
            new_beacons = repositioned[orientation]
            new_pos = reference - self.orientations[orientation][i]
            return Scanner(new_beacons, new_pos)


@dataclass
class BeaconMap:
    scanners: list[Scanner]

    @classmethod
    def from_text(cls, text: str) -> BeaconMap:
        scanners = [
            Scanner.from_lines(sc.splitlines()[1:]) for sc in text.split("\n\n")
        ]
        return cls(scanners)

    @cached_property
    def positioned_scanners(self) -> list[Scanner]:
        to_position = deque(self.scanners)
        positioned = [to_position.popleft()]
        while to_position:
            scanner = to_position.popleft()
            for other in positioned:
                if placed := scanner & other:
                    positioned.append(placed)
                    break
            else:
                to_position.append(scanner)
        return positioned

    @cached_property
    def positioned_beacons(self) -> set[np.array]:
        return {pos for s in self.positioned_scanners for pos in zip(*s.beacons.T)}


test_map = BeaconMap.from_text(
    """\
--- scanner 0 ---\n404,-588,-901\n528,-643,409\n-838,591,734\n390,-675,-793
-537,-823,-458\n-485,-357,347\n-345,-311,381\n-661,-816,-575\n-876,649,763
-618,-824,-621\n553,345,-567\n474,580,667\n-447,-329,318\n-584,868,-557
544,-627,-890\n564,392,-477\n455,729,728\n-892,524,684\n-689,845,-530
423,-701,434\n7,-33,-71\n630,319,-379\n443,580,662\n-789,900,-551\n459,-707,401

--- scanner 1 ---\n686,422,578\n605,423,415\n515,917,-361\n-336,658,858
95,138,22\n-476,619,847\n-340,-569,-846\n567,-361,727\n-460,603,-452
669,-402,600\n729,430,532\n-500,-761,534\n-322,571,750\n-466,-666,-811
-429,-592,574\n-355,545,-477\n703,-491,-529\n-328,-685,520\n413,935,-424
-391,539,-444\n586,-435,557\n-364,-763,-893\n807,-499,-711\n755,-354,-619
553,889,-390

--- scanner 2 ---\n649,640,665\n682,-795,504\n-784,533,-524\n-644,584,-595
-588,-843,648\n-30,6,44\n-674,560,763\n500,723,-460\n609,671,-379\n-555,-800,653
-675,-892,-343\n697,-426,-610\n578,704,681\n493,664,-388\n-671,-858,530
-667,343,800\n571,-461,-707\n-138,-166,112\n-889,563,-600\n646,-828,498
640,759,510\n-630,509,768\n-681,-892,-333\n673,-379,-804\n-742,-814,-386
577,-820,562

--- scanner 3 ---\n-589,542,597\n605,-692,669\n-500,565,-823\n-660,373,557
-458,-679,-417\n-488,449,543\n-626,468,-788\n338,-750,-386\n528,-832,-391
562,-778,733\n-938,-730,414\n543,643,-506\n-524,371,-870\n407,773,750
-104,29,83\n378,-903,-323\n-778,-728,485\n426,699,580\n-438,-605,-362
-469,-447,-387\n509,732,623\n647,635,-688\n-868,-804,481\n614,-800,639
595,780,-596

--- scanner 4 ---\n727,592,562\n-293,-554,779\n441,611,-461\n-714,465,-776
-743,427,-804\n-660,-479,-426\n832,-632,460\n927,-485,-438\n408,393,-506
466,436,-512\n110,16,151\n-258,-428,682\n-393,719,612\n-211,-452,876
808,-476,-593\n-575,615,604\n-485,667,467\n-680,325,-822\n-627,-443,-432
872,-547,-609\n833,512,582\n807,604,487\n839,-516,451\n891,-625,532
-652,-548,-490\n30,-46,-14
"""
)

assert len(test_map.positioned_beacons) == 79


In [2]:
import aocd

beacon_map = BeaconMap.from_text(aocd.get_data(day=19, year=2021))
print("Part 1:", len(beacon_map.positioned_beacons))


Part 1: 400


# Part 2

For part two, we only need to know the maximum Manhattan distance between the scanner positions. That's trivial to produce, all we need to do is use the `scipy.distance.pdist()` function again, this time on all scanner positions and with the `"cityblock"` metric instead of the default `euclidian` metric, and take the maximum value.


In [3]:
def max_distance(scanners: list[Scanner]) -> int:
    return int(pdist(np.array([s.position for s in scanners]), "cityblock").max())


assert max_distance(test_map.positioned_scanners) == 3621


In [4]:
print("Part 2:", max_distance(beacon_map.positioned_scanners))


Part 2: 12168
